# Accused detecation

## Appendex :
### we will be solving the problem using facial recognation method as i mentioned we will following process


## STEP1
### Lets start by importing the all necessary libraries 


In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import smtplib
from email.message import EmailMessage
import imghdr
import time

## STEP2

### Now create a folder and upload the images 

In [ ]:
#### we will be getting the images from the accused database as we are protoypeing the project we will be 
#### uploading our own photos in the folder but in the case of producation we will be requisting the data from the database

!mkdir Training_images

## STEP3
### lets load the images 

In [ ]:
path = 'Training_images'
images = []
classNames = []
myList = os.listdir(path)
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

## STEP 4
### Lets create all necessary funcations

In [ ]:
# we are using this funcation to get the encodings of the faces
def findEncodings(images):
    encodeList = []


    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown = findEncodings(images)
print('Encoding Completed')

In [ ]:
# we are using this funcation to alerting the owner at the producation level we can use IOT 

''' 
We are alerting the owner with a email here we are getting the image of accused from our model and mailing them
'''
def email_me(frame,mail,time):
    Sender_Email = "email@gmail.com" # update with your mail
    Reciever_Email = mail
    Password = 'PASSWORD'   # update your password
    img = frame
    newMessage = EmailMessage()                         
    newMessage['Subject'] = "Accused found !!!!!!!" 
    newMessage['From'] = Sender_Email                   
    newMessage['To'] = Reciever_Email                   
    newMessage.set_content(f'The following accused found at Time {time} at area Hyderabad here is the picture of accused ') 

    with open(img, 'rb') as f:
        image_data = f.read()
        image_type = imghdr.what(f.name)
        image_name = f.name

    newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        
        smtp.login(Sender_Email, Password)              
        smtp.send_message(newMessage)
    print('success msg went')


In [ ]:
# this is the main processing funcation 

# STEP1 we are extracting the encodings of each image in our folder 
# STEP2 we are storing the encodings of each image with ther name in a list
# STEP3 while the function is called we will be comapring the faces detecated in the frame with your accused data if we find 
# the match we will bw alerting the owner
def atn(frame,eml):
    email = eml

    
    
    
    imgS = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = classNames[matchIndex].upper()

            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name,frame,email)
            
    return frame

## STEP 5
### Lets run the main code 

In [ ]:
cap = cv2.VideoCapture(1)
while(cap.isOpened()):
    ret, frame = cap.read()
    frame = atn(frame,eml)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print('Execution completed')
        break
cap.release()
cv2.destroyAllWindows()